# DAT210x - Programming with Python for DS

## Module5- Lab5

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

matplotlib.style.use('ggplot') # Look Pretty

### A Convenience Function

In [2]:
def plotDecisionBoundary(model, X, y):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    padding = 0.6
    resolution = 0.0025
    colors = ['royalblue','forestgreen','ghostwhite']

    # Calculate the boundaris
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    x_range = x_max - x_min
    y_range = y_max - y_min
    x_min -= x_range * padding
    y_min -= y_range * padding
    x_max += x_range * padding
    y_max += y_range * padding

    # Create a 2D Grid Matrix. The values stored in the matrix
    # are the predictions of the class at at said location
    xx, yy = np.meshgrid(np.arange(x_min, x_max, resolution),
                       np.arange(y_min, y_max, resolution))

    # What class does the classifier say?
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the contour map
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.terrain)

    # Plot the test original points as well...
    for label in range(len(np.unique(y))):
        indices = np.where(y == label)
        plt.scatter(X[indices, 0], X[indices, 1], c=colors[label], label=str(label), alpha=0.8)

    p = model.get_params()
    plt.axis('tight')
    plt.title('K = ' + str(p['n_neighbors']))

### The Assignment

Load up the dataset into a variable called `X`. Check `.head` and `dtypes` to make sure you're loading your data properly--don't fail on the 1st step!

In [3]:
X = pd.read_csv('Datasets/wheat.data', index_col='id')
X.head()

,area,perimeter,compactness,length,width,asymmetry,groove,wheat_type
id,,,,,,,,
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,kama
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,kama
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,kama
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,kama
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,kama


In [4]:
X.dtypes

area           float64
perimeter      float64
compactness    float64
length         float64
width          float64
asymmetry      float64
groove         float64
wheat_type      object
dtype: object

Copy the `wheat_type` series slice out of `X`, and into a series called `y`. Then drop the original `wheat_type` column from the `X`:

In [5]:
y = X.wheat_type.copy()
X.drop(columns='wheat_type', inplace=True)

In [6]:
X.head()

,area,perimeter,compactness,length,width,asymmetry,groove
id,,,,,,,
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175


In [7]:
y.head()

id
0    kama
1    kama
2    kama
3    kama
4    kama
Name: wheat_type, dtype: object

Do a quick, "ordinal" conversion of `y`. In actuality our classification isn't ordinal, but just as an experiment...

In [8]:
y = y.astype(pd.api.types.CategoricalDtype(ordered=True)).cat.codes

In [9]:
y.unique()

array([1, 0, 2])

Do some basic nan munging. Fill each row's nans with the mean of the feature:

In [10]:
X.isnull().sum()

area           0
perimeter      0
compactness    3
length         0
width          1
asymmetry      0
groove         4
dtype: int64

In [11]:
X.groove.mean()

5.407529126213593

In [12]:
X[X.groove.isnull() == True]

,area,perimeter,compactness,length,width,asymmetry,groove
id,,,,,,,
7,14.11,14.10,0.8911,5.420,3.302,2.7,NaN
60,11.42,12.86,0.8683,5.008,2.850,2.7,NaN
135,15.38,14.66,0.8990,5.477,3.465,3.6,NaN
201,12.67,13.32,0.8977,4.984,3.135,2.3,NaN


In [13]:
a = X.fillna(X.groove.mean(axis=0), axis=0)
a.iloc[135]

area           15.380000
perimeter      14.660000
compactness     0.899000
length          5.477000
width           3.465000
asymmetry       3.600000
groove          5.407529
Name: 135, dtype: float64

In [14]:
for column in X.columns:
    X.fillna(X[column].mean(axis=0), axis=0, inplace=True)
    #print(type(column))

In [15]:
X.isnull().sum()

area           0
perimeter      0
compactness    0
length         0
width          0
asymmetry      0
groove         0
dtype: int64

Split `X` into training and testing data sets using `train_test_split()`. Use `0.33` test size, and use `random_state=1`. This is important so that your answers are verifiable. In the real world, you wouldn't specify a random_state:

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

Create an instance of SKLearn's Normalizer class and then train it using its .fit() method against your _training_ data. The reason you only fit against your training data is because in a real-world situation, you'll only have your training data to train with! In this lab setting, you have both train+test data; but in the wild, you'll only have your training data, and then unlabeled data you want to apply your models to.

In [18]:
from sklearn.preprocessing import Normalizer

In [19]:
normalizer = Normalizer()
normalizer.fit_transform(X_train)

array([[0.61015391, 0.65753976, 0.04229554, 0.25730029, 0.14494209,
        0.21597201, 0.24435467],
       [0.69052897, 0.601205  , 0.03190142, 0.23497673, 0.13267344,
        0.18462717, 0.23195065],
       [0.70423697, 0.60494507, 0.03269278, 0.22965848, 0.14029577,
        0.11341341, 0.22745199],
       [0.63972592, 0.65868437, 0.043794  , 0.24976533, 0.15560507,
        0.11963264, 0.23250829],
       [0.67803761, 0.63741585, 0.03914382, 0.23884728, 0.15185622,
        0.0691002 , 0.21806104],
       [0.65627803, 0.64851953, 0.0408508 , 0.24580761, 0.15115388,
        0.11236137, 0.22618317],
       [0.58390096, 0.67076651, 0.04308326, 0.26599362, 0.14160626,
        0.20806612, 0.27051679],
       [0.57029392, 0.64682687, 0.04174749, 0.25512629, 0.140475  ,
        0.33156049, 0.24470794],
       [0.65790937, 0.64589885, 0.0392299 , 0.24666041, 0.14639484,
        0.12028309, 0.22735462],
       [0.47406244, 0.54829286, 0.62621471, 0.03525523, 0.21467774,
        0.11450885, 0.1

With your trained pre-processor, transform both your training AND testing data. Any testing data has to be transformed with your preprocessor that has ben fit against your training data, so that it exist in the same feature-space as the original data used to train your models.

Just like your preprocessing transformation, create a PCA transformation as well. Fit it against your training data, and then project your training and testing features into PCA space using the PCA model's `.transform()` method. This has to be done because the only way to visualize the decision boundary in 2D would be if your KNN algo ran in 2D as well:

In [20]:
y_train.shape

(140,)

In [21]:
from sklearn.decomposition import PCA

In [22]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [23]:
X_pca.shape

(140, 2)

Create and train a KNeighborsClassifier. Start with `K=9` neighbors. Be sure train your classifier against the pre-processed, PCA- transformed training data above! You do not, of course, need to transform your labels.

In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_pca, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [ ]:
# I hope your KNeighbors classifier model from earlier was named 'knn'
# If not, adjust the following line:
plotDecisionBoundary(knn, X_pca, y_train)

Display the accuracy score of your test data/labels, computed by your KNeighbors model. You do NOT have to run `.predict` before calling `.score`, since `.score` will take care of running your predictions for you automatically.

In [ ]:
knn.score(X_test, y_test)

### Bonus

Instead of the ordinal conversion, try and get this assignment working with a proper Pandas get_dummies for feature encoding. You might have to update some of the `plotDecisionBoundary()` code.

In [ ]:
plt.show()